In [1]:
import httpx
from prefect import flow, task


@task(retries=2)
def get_repo_info(repo_owner: str, repo_name: str):
    """Get info about a repo - will retry twice after failing"""
    url = f"https://api.github.com/repos/{repo_owner}/{repo_name}"
    api_response = httpx.get(url)
    api_response.raise_for_status()
    repo_info = api_response.json()
    return repo_info


@task
def get_contributors(repo_info: dict):
    """Get contributors for a repo"""
    contributors_url = repo_info["contributors_url"]
    response = httpx.get(contributors_url)
    response.raise_for_status()
    contributors = response.json()
    return contributors


@flow(log_prints=True)
def repo_info(repo_owner: str = "PrefectHQ", repo_name: str = "prefect"):
    """
    Given a GitHub repository, logs the number of stargazers
    and contributors for that repo.
    """
    repo_info = get_repo_info(repo_owner, repo_name)
    print(f"Stars 🌠 : {repo_info['stargazers_count']}")

    contributors = get_contributors(repo_info)
    print(f"Number of contributors 👷: {len(contributors)}")


if __name__ == "__main__":
    repo_info()


15:01:04.159 | INFO    | prefect.engine - Created flow run 'eminent-duck' for flow 'repo-info'

15:01:04.231 | INFO    | Flow run 'eminent-duck' - Created task run 'get_repo_info-0' for task 'get_repo_info'

15:01:04.233 | INFO    | Flow run 'eminent-duck' - Executing 'get_repo_info-0' immediately...

15:01:05.927 | INFO    | Task run 'get_repo_info-0' - Finished in state Completed()

15:01:05.929 | INFO    | Flow run 'eminent-duck' - Stars 🌠 : 13920

15:01:05.956 | INFO    | Flow run 'eminent-duck' - Created task run 'get_contributors-0' for task 'get_contributors'

15:01:05.957 | INFO    | Flow run 'eminent-duck' - Executing 'get_contributors-0' immediately...

15:01:07.686 | INFO    | Task run 'get_contributors-0' - Finished in state Completed()

15:01:07.688 | INFO    | Flow run 'eminent-duck' - Number of contributors 👷: 30

15:01:07.719 | INFO    | Flow run 'eminent-duck' - Finished in state Completed('All states completed.')